# Extracting MIMIC Data

## Environment Setup

In [19]:
import time
import pymysql
import getpass
import pickle as pkl
import pandas as pd
from _collections import OrderedDict

## Query List

#### Construct a list of queries that we will want to run.

In [25]:
queryList = OrderedDict([('PNA', './queries/PNA-Mimic.sql'), ('CHF','./queries/CHF-Mimic.sql'), ('COPD','./queries/CHF-Mimic.sql')])

In [16]:
queryList = OrderedDict([('PNA', './queries/PNA-local.sql'), ('CHF','./queries/CHF-local.sql'), ('COPD','./queries/COPD-local.sql')])

## MIMIC Database Connection

#### Make a connection to the MIMIC database and get a cursor for record processing.

In [26]:
conn = pymysql.connect(host="mysql", 
                       port = 3306, user="jovyan", 
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"), db='mimic2')
cur = conn.cursor()

Enter MySQL passwd for jovyan········


In [21]:
conn = pymysql.connect(host="localhost", 
                       port = 3306, user="jferraro", 
                       passwd=getpass.getpass("Enter MySQL passwd "), db='mimic')
cur = conn.cursor()

Enter MySQL passwd ········


OperationalError: (2003, "Can't connect to MySQL server on 'localhost' ([Errno 99] Cannot assign requested address)")

## Retrieve our Data

#### For each query we will retrieve the data and build a ordered dictionary containing our data. Were use an ordered dictionary because we want to keep the Pneumonia, Congestive Heart Failure, and COPD cases together.

In [27]:
queries = []

for key in queryList:
    count = 1
    file = open(queryList[key], 'r')
    query = file.read()
    print("execute query: " + key)
    %time cur.execute(query)
    queries.append(query)

corpus = pd.concat([pd.read_sql_query(q, conn) for q in queries])
print(corpus.head())

conn.close()

execute query: PNA
CPU times: user 24.1 ms, sys: 12.5 ms, total: 36.6 ms
Wall time: 818 ms
execute query: CHF
CPU times: user 28.5 ms, sys: 0 ns, total: 28.5 ms
Wall time: 355 ms
execute query: COPD
CPU times: user 17.7 ms, sys: 12 ms, total: 29.6 ms
Wall time: 356 ms
                                                text label
0  \n\n\n     DATE: [**2996-12-2**] 10:25 AM\n   ...   PNA
1  \n\n\n     DATE: [**2850-2-14**] 10:22 PM\n   ...   PNA
2  \n\n\n     DATE: [**2631-10-3**] 9:52 AM\n    ...   PNA
3  \n\n\n     DATE: [**2584-11-21**] 11:17 AM\n  ...   PNA
4  \n\n\n     DATE: [**2584-11-21**] 11:17 AM\n  ...   PNA


## Safestore the Data

#### We will serialize the ordered dictionary structure out to disk. This way we will not have to rebuild this data structure when we want to use our data. It is a convenience thing....

In [29]:
file = open('differential-corpus.pkl','wb')
pkl.dump(corpus, file)
file.close()
print("Done!")

Done!


In [30]:
corpus['label']

0      PNA
1      PNA
2      PNA
3      PNA
4      PNA
5      PNA
6      PNA
7      PNA
8      PNA
9      PNA
10     PNA
11     PNA
12     PNA
13     PNA
14     PNA
15     PNA
16     PNA
17     PNA
18     PNA
19     PNA
20     PNA
21     PNA
22     PNA
23     PNA
24     PNA
25     PNA
26     PNA
27     PNA
28     PNA
29     PNA
      ... 
470    CHF
471    CHF
472    CHF
473    CHF
474    CHF
475    CHF
476    CHF
477    CHF
478    CHF
479    CHF
480    CHF
481    CHF
482    CHF
483    CHF
484    CHF
485    CHF
486    CHF
487    CHF
488    CHF
489    CHF
490    CHF
491    CHF
492    CHF
493    CHF
494    CHF
495    CHF
496    CHF
497    CHF
498    CHF
499    CHF
Name: label, Length: 1500, dtype: object